In [ ]:
from PIL import Image
import os

# Set image size
img_size = (299, 299)

# Resize images in abandoned houses folder
for filename in os.listdir('abandoned_houses'):
    img = Image.open(os.path.join('abandoned_houses', filename))
    img = img.resize(img_size)
    img.save(os.path.join('abandoned_houses_resized', filename))

# Resize images in regular houses folder
for filename in os.listdir('regular_houses'):
    img = Image.open(os.path.join('regular_houses', filename))
    img = img.resize(img_size)
    img.save(os.path.join('regular_houses_resized', filename))


In [1]:
import tensorflow as tf
import numpy as np
import os

class HouseDataset:
    def __init__(self, data_dir, img_size):
        self.data_dir = data_dir
        self.img_size = img_size
        self.class_names = ['abandoned', 'regular']

    def load_data(self):
        abandoned_files = [os.path.join(self.data_dir, 'abandoned_houses_resized', f) for f in os.listdir(os.path.join(self.data_dir, 'abandoned_houses_resized'))]
        regular_files = [os.path.join(self.data_dir, 'regular_houses_resized', f) for f in os.listdir(os.path.join(self.data_dir, 'regular_houses_resized'))]

        all_files = abandoned_files + regular_files
        labels = [0] * len(abandoned_files) + [1] * len(regular_files)

        return all_files, labels

    def preprocess_image(self, img_path):
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, self.img_size)
        img = img / 255.0
        return img

    def create_dataset(self):
        files, labels = self.load_data()
        dataset = tf.data.Dataset.from_tensor_slices((files, labels))
        dataset = dataset.map(lambda x, y: (self.preprocess_image(x), y))
        return dataset.batch(32)  # batch size


In [2]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [3]:
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=x)


In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
dataset = HouseDataset('path/to/data', (299, 299)).create_dataset()
history = model.fit(dataset, epochs=10, validation_data=dataset)


In [ ]:
from tensorflow.keras.metrics import Precision, Recall, F1Score, AUC

precision = Precision()
recall = Recall()
f1_score = F1Score()
auc = AUC()

test_dataset = HouseDataset('path/to/test/data', (299, 299)).create_dataset()

model.evaluate(test_dataset)
y_pred = model.predict(test_dataset)
y_true = [y for x, y in test_dataset]

precision.update_state(y_true, y_pred)
recall.update_state(y_true, y_pred)
f1_score.update_state(y_true, y_pred)
auc.update_state(y_true, y_pred)

print(f'Test precision: {precision.result().numpy():.3f}')
print(f'Test recall: {recall.result().numpy():.3f}')
print(f'Test F1-score: {f1_score.result().numpy():.3f}')
print(f'Test AUC-ROC: {auc.result().numpy():.3f}')

In [ ]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Set image size
img_size = (299, 299)

# Create dataset class
class HouseDataset:
    def __init__(self, data_dir, img_size):
        self.data_dir = data_dir
        self.img_size = img_size
        self.class_names = ['abandoned', 'regular']

    def load_data(self):
        abandoned_files = [os.path.join(self.data_dir, 'abandoned_houses', f) for f in os.listdir(os.path.join(self.data_dir, 'abandoned_houses'))]
        regular_files = [os.path.join(self.data_dir, 'regular_houses', f) for f in os.listdir(os.path.join(self.data_dir, 'regular_houses'))]

        all_files = abandoned_files + regular_files
        labels = [0] * len(abandoned_files) + [1] * len(regular_files)

        return all_files, labels

    def preprocess_image(self, img_path):
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, self.img_size)
        img = img / 255.0
        return img

    def create_dataset(self, files, labels, batch_size=32):
        dataset = tf.data.Dataset.from_tensor_slices((files, labels))
        dataset = dataset.map(lambda x, y: (self.preprocess_image(x), y))
        dataset = dataset.batch(batch_size)
        return dataset

# Load data
'''
supposed data structure
data/
abandoned_houses/
image1.jpg
image2.jpg
...
regular_houses/
image1.jpg
image2.jpg
...
'''
data_dir = 'path/to/data'
dataset = HouseDataset(data_dir, img_size)
all_files, labels = dataset.load_data()

# Split data into train, test, and dev sets
train_files, val_files, train_labels, val_labels = train_test_split(all_files, labels, test_size=0.2, random_state=42)
val_files, test_files, val_labels, test_labels = train_test_split(val_files, val_labels, test_size=0.5, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_dataset = dataset.create_dataset(train_files, train_labels, batch_size=32)
val_dataset = dataset.create_dataset(val_files, val_labels, batch_size=32)
test_dataset = dataset.create_dataset(test_files, test_labels, batch_size=32)

# Load pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze some layers and add a new classification head
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset)
print(f'Test loss: {test_loss:.3f}, Test accuracy: {test_acc:.3f}')

# Get predictions
y_pred = model.predict(test_dataset)
y_pred_class = tf.argmax(y_pred, axis=1)

# Evaluate metrics
y_true = test_labels
accuracy = accuracy_score(y_true, y_pred_class)
print(f'Test accuracy: {accuracy:.3f}')
print('Classification Report:')
print(classification_report(y_true, y_pred_class))
print('Confusion Matrix:')
print(confusion_matrix(y_true, y_pred_class))